In [3]:
pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 6.2 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 12.1 MB/s eta 0:00:0031m13.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 15.4 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [torchaudio] 6/7 [torchaudio]hvision]
Note: you may need to restart the kernel to use updated packages.


In [30]:
import torch 
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder



df_tracks = pd.read_csv("high_popularity_spotify_data.csv")
#df_tracks = pd.read_csv("low_popularity_spotify_data.csv", index_col = 0)

# Load both datasets

#features are energy, tempo, danceability, loudness, liveness, valence, time_signature, speechiness

predictor_features = ["energy", "tempo", "danceability", "loudness",
                     "liveness", "valence",  "time_signature",
                    "speechiness"]

# TO DO - split the data into the predictor features and the target variable (genre)
X = df_tracks[predictor_features]
y = df_tracks["playlist_genre"]

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Convert from pandas DataFrame/Series to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32)


label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

X_train = X_train.unsqueeze(1)
X_test = X_test.unsqueeze(1)

device = torch.device("cude" if torch.cuda.is_available() else "cpu")
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)




#Now that data is split into training and test we want to use a Vanilla RNN model in order to train 
class VanillaRNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_classes=10):
        super(VanillaRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        #batch_first=True means input tensors are shaped like (batch_size, sequence_length, input_size)
        self.fc = nn.Linear(hidden_size, num_classes)
        #self.fc means it's a fully connected layer that takes the last hidden state and maps it to our 10 output classes

    def forward(self, x):
        h0 = torch.zeros(1, x.shape[0], self.hidden_size, device=x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])  # last time step output
        return out

num_classes = len(np.unique(y_train.cpu().numpy()))
input_size = X_train.shape[2]

model = VanillaRNN(input_size=input_size, hidden_size=64, num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ----- Training Loop -----
epochs = 150
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# ----- Evaluation -----
model.eval()
with torch.no_grad():
    preds = model(X_test)
    predicted_classes = preds.argmax(dim=1)
    acc = (predicted_classes == y_test).float().mean()
    print(f"\n✅ Test Accuracy: {acc.item()*100:.2f}%")

# Optional: decode genre labels
predicted_genres = label_encoder.inverse_transform(predicted_classes.numpy())
  


Epoch [5/150], Loss: 3.3247
Epoch [10/150], Loss: 3.2596
Epoch [15/150], Loss: 3.1951
Epoch [20/150], Loss: 3.1297
Epoch [25/150], Loss: 3.0624
Epoch [30/150], Loss: 2.9925
Epoch [35/150], Loss: 2.9199
Epoch [40/150], Loss: 2.8451
Epoch [45/150], Loss: 2.7689
Epoch [50/150], Loss: 2.6924
Epoch [55/150], Loss: 2.6170
Epoch [60/150], Loss: 2.5442
Epoch [65/150], Loss: 2.4754
Epoch [70/150], Loss: 2.4119
Epoch [75/150], Loss: 2.3547
Epoch [80/150], Loss: 2.3043
Epoch [85/150], Loss: 2.2608
Epoch [90/150], Loss: 2.2239
Epoch [95/150], Loss: 2.1928
Epoch [100/150], Loss: 2.1666
Epoch [105/150], Loss: 2.1443
Epoch [110/150], Loss: 2.1251
Epoch [115/150], Loss: 2.1083
Epoch [120/150], Loss: 2.0935
Epoch [125/150], Loss: 2.0804
Epoch [130/150], Loss: 2.0687
Epoch [135/150], Loss: 2.0581
Epoch [140/150], Loss: 2.0484
Epoch [145/150], Loss: 2.0397
Epoch [150/150], Loss: 2.0316

✅ Test Accuracy: 37.87%
